# CatBoost

CatBoostはYandexが中心となって開発しているGBDTのOSSライブラリ

## 関連論文

[Reference papers | CatBoost](https://catboost.ai/docs/en/concepts/educational-materials-papers)

- [CatBoost: unbiased boosting with categorical features](https://arxiv.org/abs/1706.09516)
    - CatBoost の Ordered Boosting と Ordered Categorical Features Statistics の説明
- [CatBoost: gradient boosting with categorical features support](http://learningsys.org/nips17/assets/papers/paper_11.pdf)
    - CatBoostの基本的な動作原理の説明
- [Minimal Variance Sampling in Stochastic Gradient Boosting](https://arxiv.org/abs/1910.13204)
    - CatBoostのデフォルトのサンプリング手法MVSについて
- [Finding Influential Training Samples for Gradient Boosted Decision Trees](https://arxiv.org/abs/1802.06640)
    - あるサンプルに対する予測について影響の大きい訓練データ中の事例を探索するアルゴリズムについて
- [Which Tricks are Important for Learning to Rank?](https://arxiv.org/abs/2204.01500)
    - ランキング学習の誤差関数について、LambdaMART, YetiRank, StochasticRankを比較してYetiRankの改良を提案
- [Gradient Boosting Performs Gaussian Process Inference](https://arxiv.org/abs/2206.05608)
    - 対称決定木（symmetric decision tress）の勾配ブースティングはカーネル法と等価であり、カーネルリッジ回帰問題の解へと変換できることを示した
    - つまり、CatBoostでガウス過程と同様の結果が得られることを示した。
    - 実装：[sample_gaussian_process | CatBoost](https://catboost.ai/docs/en/concepts/python-reference_sample_gaussian_process)

[Prokhorenkova, et al. (2018). CatBoost: unbiased boosting with categorical features. NeurIPS2018](https://arxiv.org/pdf/1706.09516.pdf)

[Dorogush, et al. (2018). CatBoost: gradient boosting with categorical features support. NIPS2017](http://learningsys.org/nips17/assets/papers/paper_11.pdf)

## Prediction Shift

[CatBoost論文のprediction shiftについて完全に理解する - threecourse’s blog](https://threecourse.hatenablog.com/entry/2020/05/29/204944)

- 各iterationで同じ訓練データを使うことの問題
- 1本目と2本目の木の作成で同じデータセットを使うことにより、予測値の期待値に$1 / (n − 1)$ に比例する偏りが発生する。
- prediction shiftの影響を受けないよう、CatBoostではデータ数が少ない場合にはordered boostingというアルゴリズムを使用するようになっている。

## Categorical featuresの扱い

### ラベル統計に基づく方法

別の方法として、ラベル値を用いてカテゴリごとの統計量を計算する手法がある。  
データセット $\{ (x_i, Y_i) \}$ が与えられており、$x_i$ は数値的およびカテゴリ的特徴を含むベクトル、$Y_i$ はラベルであるとする。  
最も単純な方法は、カテゴリ全体の平均ラベル値で置き換えるものである。すなわち、カテゴリ $x_{i,k}$ は次のように置き換えられる：

$$
x_{i,k} \to 
\frac{\sum_{j=1}^n [x_{j,k} = x_{i,k}] \cdot Y_j}
     {\sum_{j=1}^n [x_{j,k} = x_{i,k}]}
$$

ここで、$[\cdot]$ はイベルソン括弧（条件が真なら1、偽なら0）である。  
しかし、この方法は明らかに **過学習** を引き起こす。  
例えば、あるカテゴリがデータ中に1回しか出現しない場合、その数値はそのラベルと等しくなってしまう。

この問題を避けるための単純な方法は、データセットを2つに分割し、一方で統計量を計算し、もう一方で訓練することである。  
しかしこれはデータ量を減らしてしまう。



---

### CatBoost における戦略

CatBoost は過学習を抑制しつつ、全データを利用できる効率的な戦略を採る。  
すなわち、データセットの **ランダム順列** $\sigma$ を生成し、各サンプルについて、その順列内で前に出現する同じカテゴリを持つサンプルのラベル平均を計算する：

$$
x_{\sigma_p, k} \to
\frac{\sum_{j=1}^{p-1} [x_{\sigma_j, k} = x_{\sigma_p, k}] \cdot Y_{\sigma_j} + a \cdot P}
     {\sum_{j=1}^{p-1} [x_{\sigma_j, k} = x_{\sigma_p, k}] + a}
\tag{1}
$$

ここで、  
- $a > 0$ は **prior の重み**、  
- $P$ は **prior 値**（例えば回帰なら平均ラベル値、二値分類なら正例の事前確率）である。

prior を導入することにより、出現頻度の低いカテゴリから生じるノイズを軽減できる。  
さらに複数の順列を用いることも可能であるが、単純に統計量を併用すると過学習を起こす。  
CatBoost では次節で述べる新しい葉値計算法により、複数順列の利用を安全に行える。

---

### 特徴の組み合わせ

複数のカテゴリ特徴を組み合わせて新しい特徴を作ることもできる。  
例えば、音楽推薦タスクにおいて「ユーザーID」と「音楽ジャンル」があるとする。  
ユーザーが特定のジャンルを好む情報（例：「ユーザー123はロックが好き」）は、各特徴を別々に変換すると失われるが、2つを組み合わせることで新たな強力な特徴となる。

しかし、全ての組み合わせを考慮すると組み合わせ数が指数的に増えるため、CatBoost では **貪欲法 (greedy approach)** により木構築中の分割で生成可能な組み合わせのみを考慮する。

- 最初の分割では組み合わせを考慮しない。  
- 以降の分割では、現在の木で使われたカテゴリ特徴とデータセット中の他のカテゴリ特徴を組み合わせる。  
- 組み合わせ値は「その場で」数値に変換される。  
- 数値特徴とカテゴリ特徴の組み合わせも同様に扱われる。

---

### 実装上の重要な詳細

カテゴリを数値に置き換えるもう一つの単純で強力な方法は、**そのカテゴリの出現回数**を計算することである。  
CatBoost ではこの統計も計算し、特徴の組み合わせにも適用している。

また、CatBoost アルゴリズムの各ステップで最適な prior を適合させるために、複数の prior を試し、それぞれについて特徴を構築する。  
これは前述の標準的手法よりも品質面で優れている。

---
